In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import exists, join as pjoin, realpath
NBROOT = realpath(os.curdir)

import logging
logging.basicConfig(level=logging.DEBUG)

import sys
sys.path.append(realpath('..'))
sys.path.append(realpath('../../eslope/development/src'))

from subprocess import check_call, CalledProcessError

try:
    # like os.system but with live output
    from IPython.utils.process import system
    def check_run(cmd):  # type:ignore
        r = system(cmd)
        if r: raise CalledProcessError(r, cmd)
        return r
except ImportError:
    from os import system
    def check_run(cmd):
        return check_call(cmd, shell=True)

from IPython.display import display, Image
from PIL import Image as Img

# own
import mbt_util as M
from src import etopo_meta as E, img_util as G, geometry as GY


In [3]:
names = GY.bbalp_names
paths = {i: name + '.mbtiles' for i, name in names.items()}
mbtfold = '/opt/map/mapdata/mbtiles/'
abspaths = {i: mbtfold + p for i, p in paths.items()}

# Metadata update

In [75]:
for i, mbt in paths.items():
    if not os.path.exists(mbt):
        continue
    desc = E.DESC[i]
    attrib = E.ATTRIB[i]
    M.update_mbt_meta(mbt, desc=desc, attrib=attrib)
    print(i, 'd:', M.get_meta(mbt)['description'], '\na:', M.get_meta(mbt).get('attribution'))
    print()


1 d: Carte topographique de la zone alps1-Mercantour-Ubaye-Cuneese, fusion de cartes IGN et Bugianen:
* IGN Top 25 et Top 100 (2021). https://www.geoportail.gouv.fr
* Bugianen 2205 (2022) : Mappa escursionistica delle alpi piemontesi, valdostane e dintorni.
Per la legenda e l'origine dei dati visita <a href="https://tartamillo.wordpress.com/bugianen/
 
a: © <a href="https://geoservices.ign.fr/cgu-licences">IGN 2021</a> / © <a href="https://tartamillo.wordpress.com/bugianen/">CC BY-NC-SA 3.0 Maki</a>

2 d: Carte topographique de la zone alps2-Gap-Digne-Toulon-Apt: IGN Top 25 et Top 100 (2021). https://www.geoportail.gouv.fr 
a: © <a href="https://geoservices.ign.fr/cgu-licences">IGN 2021</a>

3 d: Carte topographique de la zone alps3-Vercors-Ecrins-Queyras-Cozie, fusion de cartes IGN et Bugianen:
* IGN Top 25 et Top 100 (2021). https://www.geoportail.gouv.fr
* Bugianen 2205 (2022) : Mappa escursionistica delle alpi piemontesi, valdostane e dintorni.
Per la legenda e l'origine dei dati v

In [4]:
for i, mbt in abspaths.items():
    if i != 8:
        continue
    if not os.path.exists(mbt):
        print(mbt, '?')
        continue
    desc = E.DESC[i]
    attrib = E.ATTRIB[i]
    M.update_mbt_meta(mbt, desc=desc, attrib=attrib)
    print(i, 'd:', M.get_meta(mbt)['description'], '\na:', M.get_meta(mbt).get('attribution'))
    print()


8 d: Carte topographique de la zone alps8-Lombardia-Como-StMoritz-Adamello, fusion de cartes SwissTopo et Kompass:
* SwissTopo mix-scale (2021): Switzerland SwissTopo national map. https://map.geo.admin.ch
Unlike the website, this is a downscaled version when possible (1 zoom level up for each layer).
* Kompass (2021). https://www.kompass.de/wanderkarte/
 
a: © <a href="https://shop.swisstopo.admin.ch/fr/geodonnees-gratuites">SwissTopo OGD</a> / © <a href="https://www.kompass.de/b2b/lizenzen/">Kompass.de</a>



# Finally, upload to GH

In [5]:
N = 8
p = abspaths[N]
system('gh release delete-asset -y 202301 ' + paths[N])
system('gh release upload 202301 ' + p)


asset alps8-Lombardia-Como-StMoritz-Adamello.mbtiles not found in release 202301
Successfully uploaded 1 asset to 202301


0

In [10]:
for i, p in abspaths.items():
#    if i >= 6:
        system('gh release delete-asset -y 202301 ' + paths[i])
        system('gh release upload 202301 ' + p)
        # check_run('gh release upload 202301 ' + p)

✓ Deleted asset alps6-Aoste-Martigny-Monch-MtRosa.mbtiles from release 202301
Successfully uploaded 1 asset to 202301
✓ Deleted asset alps7-Ticino-Finsteraarhorn-Lorenzhorn.mbtiles from release 202301
Successfully uploaded 1 asset to 202301
asset alps8-Como-Mesolcina-Adamello-Ortler.mbtiles not found in release 202301
Successfully uploaded 1 asset to 202301
asset alps9-CHCW-Vaud-Luzern-Grimselpass.mbtiles not found in release 202301
Successfully uploaded 1 asset to 202301
asset alps10-CHCE-Realp-Pfunds.mbtiles not found in release 202301
Successfully uploaded 1 asset to 202301
asset alps11-CHNE-Zurich-LH-Lechtal.mbtiles not found in release 202301
stat ../../mapdata/mbtiles/alps11-CHNE-Zurich-LH-Lechtal.mbtiles: no such file or directory
asset alps12-Arco-Innsbruck-Bruneck-Marmolada.mbtiles not found in release 202301
Successfully uploaded 1 asset to 202301
asset alps13-Civetta--Unterwossen.mbtiles not found in release 202301
stat ../../mapdata/mbtiles/alps13-Civetta--Unterwossen.mbtil

In [8]:
system('gh release delete-asset -y 202301 ' + paths[-1])
system('gh release upload 202301 ' + abspaths[-1])

asset alps-z612.mbtiles not found in release 202301
Successfully uploaded 1 asset to 202301


0

In [7]:
M.real_bounds(abspaths[1], log=print)

real bounds  10 [6.33, 43.58, 8.09, 44.59] 530 650 534 653
real bounds  11 [6.33, 43.58, 7.91, 44.59] 1060 1300 1068 1307
real bounds  12 [6.33, 43.58, 8.09, 44.59] 2120 2600 2139 2615
real bounds  13 [6.33, 43.58, 8.09, 44.59] 4240 5200 4279 5231
real bounds  14 [6.33, 43.58, 8.09, 44.59] 8480 10400 8559 10463
real bounds  15 [6.33, 43.58, 8.09, 44.59] 16960 20800 17119 20927
real bounds  16 [6.33, 43.58, 8.09, 44.59] 33920 41600 34239 41855


(10,
 16,
 LngLatBbox(west=6.328124999999996, south=43.580390855607845, east=8.085937500000016, north=44.590467181308846))

In [1]:
GY.etopo2geojson(NBROOT+'/../data/areas/etopo_alps_extents.geojson')
GY.MultiPolygon(map(GY.bb2poly, GY.bbalps.values()))

NameError: name 'GY' is not defined